In [1]:

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import pathlib
import random
import glob

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
from keras.preprocessing import image_dataset_from_directory
import tensorflow as tf
from tensorflow.keras import layers
import PIL

import matplotlib.pyplot as plt
import matplotlib.image as mpimg

images_path = os.path.join(os.path.realpath('..'), "input", "kermany2018", "OCT2017 ")
images_path = pathlib.Path(images_path)

image_count = len(list(images_path.glob('*/*.jpeg')))
print(image_count)

batch_size = 32
img_height = 180
img_width = 180

test_ds = image_dataset_from_directory(
os.path.join(images_path, "test"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

0
Found 968 files belonging to 4 classes.


In [2]:
val_ds = image_dataset_from_directory(
os.path.join(images_path, "val"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 32 files belonging to 4 classes.


In [3]:

train_ds = image_dataset_from_directory(
  os.path.join(images_path, "train"),
  validation_split=0,
  seed=123,
  image_size=(img_height, img_width),
  batch_size=batch_size)

Found 83484 files belonging to 4 classes.


In [4]:
num_classes = 4

model = tf.keras.Sequential([
  layers.experimental.preprocessing.Rescaling(1./255),
  layers.Conv2D(32, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(64, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Conv2D(128, 3, activation='relu'),
  layers.MaxPooling2D(),
  layers.Flatten(),
  layers.Dense(128, activation='relu'),
  layers.Dense(num_classes)
])

In [5]:

model.compile(
  optimizer='adam',
  loss=tf.losses.SparseCategoricalCrossentropy(from_logits=True),
  metrics=['accuracy'])

In [6]:
model.fit(
  train_ds,
  validation_data=val_ds,
  epochs=1
)

model.summary()

2609/2609 [==============================] - 437s 168ms/step - loss: 0.4963 - accuracy: 0.8194 - val_loss: 0.1261 - val_accuracy: 0.9688
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
rescaling (Rescaling)        (None, 180, 180, 3)       0         
_________________________________________________________________
conv2d (Conv2D)              (None, 178, 178, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 89, 89, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 87, 87, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 43, 43, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 41, 41, 128)   

In [7]:
model.evaluate(test_ds)

31/31 [==============================] - 3s 102ms/step - loss: 0.1642 - accuracy: 0.9504


[0.16421499848365784, 0.9504132270812988]

In [8]:
# Calculate F1 score
from sklearn import metrics
# metrics.f1_score(y_actual, y_pred, labels=[0, 1, 2], average='macro')
y_pred =[]
y_true = []

i = 0
for batch, lbls in test_ds.as_numpy_iterator():
    prediction = model.predict_on_batch(batch)
    
    for i in range(len(prediction)):
        y_true.append(lbls[i])
        prediction_single = np.argmax(prediction[i])
        y_pred.append(prediction_single)
#     print(batch.shape)
#     print(lbls.shape)
print(y_true)
print(y_pred)
    
print(metrics.f1_score(y_true, y_pred, labels=[0, 1, 2, 3], average='macro'))

[3, 0, 1, 1, 0, 0, 1, 1, 1, 1, 0, 3, 3, 3, 1, 0, 3, 0, 2, 0, 0, 1, 3, 1, 3, 2, 2, 3, 0, 1, 1, 0, 0, 3, 0, 3, 0, 3, 1, 1, 1, 0, 2, 3, 2, 2, 1, 0, 1, 0, 1, 0, 3, 3, 2, 2, 2, 2, 0, 3, 2, 3, 1, 3, 1, 3, 3, 3, 1, 2, 2, 2, 1, 1, 3, 1, 3, 1, 2, 3, 2, 2, 0, 2, 1, 2, 1, 1, 3, 1, 3, 1, 1, 0, 0, 3, 1, 2, 3, 1, 0, 1, 2, 1, 1, 0, 1, 1, 1, 2, 1, 3, 1, 0, 0, 3, 0, 2, 1, 0, 0, 0, 1, 0, 1, 0, 1, 1, 1, 3, 0, 2, 2, 3, 3, 2, 1, 2, 3, 3, 3, 0, 2, 1, 2, 3, 0, 0, 2, 3, 2, 0, 0, 3, 2, 3, 2, 0, 3, 3, 2, 1, 0, 3, 1, 1, 2, 2, 1, 0, 2, 1, 3, 2, 0, 2, 1, 0, 0, 0, 0, 3, 3, 1, 0, 2, 3, 0, 1, 2, 0, 1, 2, 3, 1, 1, 1, 3, 3, 2, 2, 0, 3, 1, 2, 3, 3, 2, 2, 2, 0, 2, 3, 2, 3, 3, 0, 0, 1, 3, 2, 1, 0, 0, 2, 0, 1, 3, 0, 0, 2, 0, 3, 3, 3, 1, 0, 0, 3, 0, 1, 3, 3, 3, 0, 0, 2, 1, 3, 3, 2, 2, 1, 0, 0, 2, 2, 3, 3, 0, 1, 2, 3, 0, 0, 0, 0, 1, 1, 0, 2, 0, 3, 3, 0, 2, 1, 1, 2, 2, 0, 2, 2, 1, 1, 2, 3, 3, 1, 0, 3, 2, 2, 2, 2, 1, 1, 0, 3, 1, 2, 2, 2, 3, 2, 1, 0, 0, 3, 3, 1, 2, 2, 3, 2, 3, 1, 0, 2, 2, 1, 1, 2, 1, 0, 3, 2, 2, 0, 0, 0, 3, 1, 